# Karger algorithm

## Support objects

In [155]:
import copy
import sys
import time
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.99 s


In [156]:
class Graph:
    def __init__(self):
        self.n_nodes, self.n_edges = 0, 0
        self.nodes = defaultdict() # adjacency lists
        self.edges = []

    def addEdge(self, v1:int, v2:int):
        self.nodes[v1].append(v2)
        #self.nodes[v2].append(v1) # Graph is undirected
        if (v2, v1) not in (self.edges):
          self.edges.append((v1, v2))
          self.n_edges += 1

    def buildGraph(self):
        self.n_nodes = len(dataset)
        for i in range (1, len(dataset) + 1):
          self.nodes[i] = []
        for v in dataset:
          for i in range (1, len(v)):
            self.addEdge(v[0], v[i])
                

time: 14.1 ms


## Reading the input

In [157]:
from collections import defaultdict
def readInput(path: str):
    lines = open(path, "r").read().split("\n")
    input = []
    for line in range(len(lines)-1):
        input.append(list(map(int, lines[line].split()))) # Transform list of strings to list of integers

    return input

time: 6.65 ms


## Deep copy

In [158]:
def graphCopy(graph: Graph):
    newGraph = Graph()
    newGraph.n_nodes, newGraph.n_edges = graph.n_nodes, graph.n_edges
    newGraph.nodes = copy.deepcopy(graph.nodes)
    newGraph.edges = copy.deepcopy(graph.edges)
    return newGraph

time: 3.6 ms


## Karger implementation

In [159]:
import random
import math

def fullContraction(graph):
    sum1 = 0
    sum2 = 0
    sum3 = 0
    while graph.n_nodes > 2:

        e = graph.edges[random.randrange(1, graph.n_edges)] # select a random edge

        # Contraction
        id = "Z_" + str(e[0]) + str(e[1])
        graph.nodes[id] = []

        del_edges = 0
        delete_list = []
        
        start1 = time.time()
        for i in range(0, len(graph.edges)):
          if graph.edges[i][0] == e[0] or graph.edges[i][0] == e[1] or graph.edges[i][1] == e[0] or graph.edges[i][1] == e[1]:
            delete_list.append(i)
            del_edges += 1

        cont = 0
        for i in delete_list:
          graph.edges.pop(i - cont)
          cont += 1

        end1 = time.time()
        sum1 = sum1 + (end1 - start1)
        
        
        add_edges = 0
        start2 = time.time()
        for v in graph.nodes[e[0]]:
          if v != e[1]:
            graph.nodes[id].append(v)
            graph.edges.append((id, v))
            graph.n_edges += 1
            graph.nodes[v].remove(e[0])
            
            graph.nodes[v].append(id)
        end2 = time.time()
        sum2 = sum2 + (end2 - start2)

        start3 = time.time()
        for v in graph.nodes[e[1]]:
          if v != e[0]:
            graph.nodes[id].append(v)
            graph.edges.append((id, v))
            add_edges += 1
            graph.nodes[v].append(id)
            graph.nodes[v].remove(e[1])
        end3 = time.time()
        sum3 = sum3 + (end3 - start3)
              

        graph.nodes[e[0]] = []
        graph.nodes[e[1]] = []

        graph.n_nodes -= 1
        graph.n_edges -= (del_edges - add_edges) 

    print("sum1: ", sum1)
    print("sum2: ", sum2)
    print("sum3: ", sum3)
    return graph.n_edges

def karger(graphs):
    min = math.inf
    for g in graphs:
        t = fullContraction(g)
        if t < min: min = t
    return min

time: 69.4 ms


In [160]:
dataset = readInput("input_random_13_50.txt")

graph = Graph()
graph.buildGraph()

k = int((graph.n_nodes**2 / 2) * math.log(graph.n_nodes))
print("k: ", k)
random.seed(random.randrange(1, 1000))

temp = [None] * k

for i in range(0, len(temp)):
  temp[i] = graphCopy(graph)


k:  4890
time: 13.5 s


In [161]:
result = karger(temp)
print("final result =", result)

Streaming output truncated to the last 5000 lines.
sum2:  0.0056836605072021484
sum3:  0.0014407634735107422
sum1:  0.009794950485229492
sum2:  0.005622148513793945
sum3:  0.0014712810516357422
sum1:  0.009867668151855469
sum2:  0.00710606575012207
sum3:  0.0022351741790771484
sum1:  0.009535074234008789
sum2:  0.006774425506591797
sum3:  0.0013570785522460938
sum1:  0.010452985763549805
sum2:  0.008152961730957031
sum3:  0.0013017654418945312
sum1:  0.010154485702514648
sum2:  0.006322383880615234
sum3:  0.002313852310180664
sum1:  0.011516571044921875
sum2:  0.006593465805053711
sum3:  0.0014488697052001953
sum1:  0.010570526123046875
sum2:  0.015009880065917969
sum3:  0.0016715526580810547
sum1:  0.013737916946411133
sum2:  0.008184671401977539
sum3:  0.002448558807373047
sum1:  0.010833740234375
sum2:  0.007931232452392578
sum3:  0.0019252300262451172
sum1:  0.009655475616455078
sum2:  0.0071642398834228516
sum3:  0.0014410018920898438
sum1:  0.009775638580322266
sum2:  0.007185220